# Optimizing analysis parameters $L$ and $\epsilon^2$
This notebooks shows different ways to optimize the analysis parameters.

In [ ]:
using DIVAnd
using Makie, CairoMakie, GeoMakie
using Dates
using Statistics
include("../config.jl")

## Data reading

In [ ]:
varname = "Salinity"
filename = salinityprovencalfile
download_check(salinityprovencalfile, salinityprovencalfileURL)

obsval,obslon,obslat,obsdepth,obstime,obsid = loadobs(Float64, filename, varname);

### Topography and grid definition

See the notebook on [bathymetry](../2-Preprocessing/06-topography.ipynb) for more explanations.

In [ ]:
dx = dy = 0.125/2.
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr, latr)

bathname = gebco04file
download_check(gebco04file, gebco04fileURL)

Reading the bathymetry and creating the mask

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr)

mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

plot_mask(bx, by, mask)

## Data selection for example

Cross validation, error calculations etc. assume independant data. Hence do not take high-resolution vertical profiles with all data but restrict yourself to specific small depth ranges. 

Here August data near the surface surface.     
We also perform a test on values to eliminate obvious outliers:

In [ ]:
datadepth=1
depthprecision=0.5

sel = (obsdepth .< datadepth) .& 
(obsdepth .>= (datadepth-depthprecision)) .& 
(Dates.month.(obstime) .== 8) .& 
(obsval .> 37)

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];
@show (size(obsval))
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

We modify data weight by taking into account close points.      
**⚠️** This operation is particularly costly when dealing with big datasets.

In [ ]:
?DIVAnd.weight_RtimesOne

In [ ]:
rdiag=1.0./DIVAnd.weight_RtimesOne((obslon,obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag)

## Analysis

Analysis `fi` using mean data as background.      
Structure `s` is stored for later use.

In [ ]:
len= 1
epsilon2 = 1
fi,s = DIVAndrun(mask, (pm,pn), (xi,yi), (obslon,obslat), obsval.-mean(obsval), len, epsilon2*rdiag);

### Compute residuals
Using the analysis and the structure `s`

In [ ]:
dataresiduals = DIVAnd_residualobs(s,fi);

### Generate some plots:
1. Analysis with data points
2. Data residuals
3. Residuals vs value

In [ ]:
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Analysis and observations", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval), interpolate = false, colorrange=[37.6, 38.2])
sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37.6, 38.2])
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false, label = "S")
fig

In [ ]:
rscale=sqrt(var(obsval))
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Residuals", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
sc = scatter!(obslon, obslat, color=dataresiduals, markersize=7, colorrange=[-rscale, rscale], 
    colormap=:Spectral)
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], sc, vertical = false, label = "S")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1]; title = "Residuals as function of value", 
    xlabel="Observation values", ylabel="Residual values")
sc = plot!(ax, obsval, dataresiduals, markersize=7)
fig

# Cross validation

Take out data and measure difference between these data points not used and the analysis. Three methods are implemented 
## Define method used
    # 1: full CV
    # 2: sampled CV
    # 3: GCV
    # 0: automatic choice between the three possible ones, default value

In [ ]:
bestfactorl=ones(4)
bestfactore=ones(4)
fig = Figure(size=(600, 600))

# Loop over the methods
for imeth=0:3

    bestfactorl[imeth+1],bestfactore[imeth+1], cvval,cvvalues, x2Ddata,y2Ddata,cvinter,xi2D,yi2D = 
    DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2*rdiag,2,3,imeth);
    
    @show bestfactorl[imeth+1],bestfactore[imeth+1];
    
    ax = Axis(fig[1+imeth÷2, 1+imeth%2]; title = "Method $imeth", 
    xlabel="Log10 scale factor L", ylabel="Log10 scale factor e2")
    heatmap!(ax, xi2D[:,1], yi2D[1,:], cvinter, interpolate=false)
    plot!(ax, xi2D[:], yi2D[:], markersize=1)
    plot!(ax, [log10.(bestfactorl[imeth+1])], [log10.(bestfactore[imeth+1])])

end
fig

<div class="alert alert-block alert-warning">
⚠️ Any tests with resulting length scales being smaller than around 4 times the grid spacing are meaningless.
</div>

### Analysis with optimized values

In [ ]:
newl=len*bestfactorl[2]
newe=epsilon2*bestfactore[2]

fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);
dataresiduals=DIVAnd_residualobs(s,fi)

In [ ]:
fig = Figure(size = (600, 370))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", 
title = "Analysis and observations with\nnew L  = $(round(newl, digits=3)) and new e2 = $(round(newe, digits=3))", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval), interpolate = false, colorrange=[37., 38.5])
sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37., 38.5])
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false, label = "S")
fig

In [ ]:
rscale=sqrt(var(obsval))
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Residuals", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
sc = scatter!(obslon, obslat, color=dataresiduals, markersize=7, colorrange=[-rscale, rscale], 
    colormap=Reverse(:Spectral))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], sc, vertical = false, label = "S")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1]; title = "Residuals as function of value\nwith the new parameter values", 
    xlabel="Observation values", ylabel="Residual values")
sc = plot!(ax, obsval, dataresiduals, markersize=7)
fig

## Only one parameter optimized

If $L$ is fixed by other calibration or method,     
you can decide to optimize only $\epsilon^2$:

In [ ]:
lenfixed=1
epsilon2=.1

fig = Figure(size=(600, 600))
for imeth=0:3
    bestfactore[imeth+1], cvval,cvvalues, x2Ddata,cvinter,xi2D = 
    DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag,0,4,imeth);
    
    ax = Axis(fig[1+imeth÷2, 1+imeth%2]; title = "Method $imeth", xlabel="Log10 scale factor e2")
    plot!(ax, xi2D, cvinter, markersize=2, color=:black)
    plot!(ax, [log10.(bestfactore[imeth+1])], [cvval], markersize=10)

end
fig

### Analysis with the optimised values

In [ ]:
newl=lenfixed
newe=epsilon2*bestfactore[3]
@show newe
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);
dataresiduals=DIVAnd_residualobs(s,fi)

In [ ]:
fig = Figure(size = (600, 370))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", 
title = "Analysis and observations with\n L  = $(lenfixed) and new e2 = $(round(newe, digits=3))", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval), interpolate = false, colorrange=[37., 38.5])
sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37., 38.5])
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false, label = "S")
fig

In [ ]:
rscale=sqrt(var(obsval))
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Residuals", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
sc = scatter!(obslon, obslat, color=dataresiduals, markersize=7, colorrange=[-rscale, rscale], 
    colormap=Reverse(:Spectral))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], sc, vertical = false, label = "S")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1]; title = "Residuals as function of value\nwith the new parameter values", 
    xlabel="Observation values", ylabel="Residual values")
sc = plot!(ax, obsval, dataresiduals, markersize=7)
fig

## Adaptive method

> G. Desroziers, L. Berre, B. Chapnik and P. Poli (2005),       
Diagnosis of observation, background and analysis-error statistics in observation space,    
_Q. J. R. Meteorol. Soc._, __131__, pp. 3385–3396      
> doi: [10.1256/qj.05.108](https://rmets.onlinelibrary.wiley.com/doi/abs/10.1256/qj.05.108).

This adaptive method activated by call with 0,0 points to sample.

In [ ]:
myiterations=7
cvbest2=zeros(myiterations);
eps2=zeros(myiterations)
epsilon2=1
for i=1:myiterations
    cvval,factor=DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag,
    0,0,3);
    eps2[i]=epsilon2;
    cvbest2[i]=cvval;
    epsilon2=epsilon2*factor
    @show epsilon2
end

Other direct application of Desroziers

In [ ]:
myiterations=7

eps2=zeros(myiterations)
epsilon2=1
for i=1:myiterations
    fit,sit=DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag);
    eps2[i]=epsilon2;
    factor=DIVAnd_adaptedeps2(sit,fit)
    epsilon2=epsilon2*factor
    @show epsilon2
end

### Analysis with the optimized value of $\epsilon^2$

In [ ]:
newl=lenfixed
newe=epsilon2
@show newe
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);
dataresiduals=DIVAnd_residualobs(s,fi);

In [ ]:
fig = Figure(size = (600, 370))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", 
title = "Analysis and observations with\n L  = $(lenfixed) and new e2 = $(round(newe, digits=3))", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval), interpolate = false, colorrange=[37., 38.5])
sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37., 38.5])
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false, label = "S")
fig

In [ ]:
rscale=sqrt(var(obsval))
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Residuals", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
sc = scatter!(obslon, obslat, color=dataresiduals, markersize=7, colorrange=[-rscale, rscale], 
    colormap=Reverse(:Spectral))
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], sc, vertical = false, label = "S")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1]; title = "Residuals as function of value\nwith the new parameter values", 
    xlabel="Observation values", ylabel="Residual values")
sc = plot!(ax, obsval, dataresiduals, markersize=7)
fig

In [ ]:
@info(var(dataresiduals),var(obsval),var(fi[(fi.!==NaN)]))

## More information

In [ ]:
?DIVAnd_cv

# Exercise

1. Redo for different data by changing the datadepth parameter introduced in the data selection part.      
(Compare surface behaviour with deeper regions by using another `datadepth` value).
2. Take out the data weight modification.
3. Possibly force the cross-validation method (use `?DIVAnd_cv`).
4. Once opimized, try to redo optimization with starting point being the first estimate.

<div class="alert alert-block alert-warning">
⚠️ Remember: optimization relies on a series of hypotheses.<br>       
In particular, data independance and isotropy is very often NOT ensured.<br>    
When in doubt, increase $\epsilon^2$ and/or check for "duplicates".
</div>